In [15]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importar nuestras funciones personalizadas
from funciones_calculohormigon import *

df=pd.read_csv('datos_lab_mejor.csv')

#preparar los datos para machine learning
#def preparar_datos(df):
#    '''
#    '''
#    df_ml=df.copy()
#    df_ml['asentamiento_num']=df_ml['asentamiento'].apply(procesar_asentamiento_str)

# Función para preparar datos para machine learning
def preparar_datos_ml(df):
    """
    Prepara el dataset para entrenar modelos de ML
    """
    # Filtrar solo datos relevantes para predicción
    df_ml = df.copy()
    
    # Limpiar asentamiento 
    df_ml['asentamiento_num'] = df_ml['asentamiento'].apply(procesar_asentamiento_str)
    
    # Filtrar filas con asentamiento válido
    df_ml = df_ml[df_ml['asentamiento_num'].notna()]
    
    # Seleccionar características para el modelo
    caracteristicas = ['densidad', 'peso_suelto', 'peso_compactado', 'diametro_maximo', 'fck', 'modulo_finura']
    
    # Crear dataset final para ML
    X = df_ml[caracteristicas].fillna(0)  # Llenar NaN con 0
    y = df_ml['asentamiento_num']
    
    print(f"DATOS LISTOS PARA ML:")
    print(f"   - Muestras: {len(X)}")
    print(f"   - Características: {list(X.columns)}")
    print(f"   - Rango asentamiento: {y.min():.1f} - {y.max():.1f} cm")
    
    return X, y, df_ml

# Preparar datos
X, y, df_ml = preparar_datos_ml(df)

# Mostrar distribución de asentamiento
print("\n📈 DISTRIBUCIÓN DE ASENTAMIENTO:")
print(y.describe())

# GRÁFICO 1: Distribución de asentamientos
fig1 = px.histogram(df_ml, x='asentamiento_num', 
                   title='Distribución de Asentamientos en el Dataset',
                   labels={'asentamiento_num': 'Asentamiento (cm)'},
                   color_discrete_sequence=["#19B9E5"])
fig1.show()



# Reemplazar NaN en modulo_finura con un valor por defecto (promedio)
modulo_finura_promedio = df_ml['modulo_finura'].mean()
df_ml_clean = df_ml.copy()
df_ml_clean['modulo_finura'] = df_ml_clean['modulo_finura'].fillna(modulo_finura_promedio)

print(f"   - NaN en modulo_finura: {df_ml['modulo_finura'].isna().sum()} → 0")
print(f"   - Valor promedio usado: {modulo_finura_promedio:.2f}")

#GRÁFICO 2: Asentamiento vs Tamaño Máximo - CON DATOS LIMPIOS
fig2 = px.scatter(df_ml_clean, x='diametro_maximo', y='asentamiento_num',
                 title='Relación: Tamaño Máximo vs Asentamiento',
                 labels={'diametro_maximo': 'Tamaño Máximo (mm)', 
                        'asentamiento_num': 'Asentamiento (cm)'},
                 color='fck', 
                 size='modulo_finura',
                 hover_data=['material'],
                 size_max=15)  # Limitar tamaño máximo

fig2.show()
#GRÁFICO 3: Mapa de calor de correlaciones
corr_matrix = df_ml[['asentamiento_num', 'densidad', 'diametro_maximo', 'fck', 'modulo_finura']].corr()

fig3 = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.columns,
                   colorscale='RdBu',
                   zmin=-1, zmax=1,
                   hoverongaps=False))
fig3.update_layout(title='Mapa de Calor - Correlaciones entre Variables')
fig3.show()



DATOS LISTOS PARA ML:
   - Muestras: 107
   - Características: ['densidad', 'peso_suelto', 'peso_compactado', 'diametro_maximo', 'fck', 'modulo_finura']
   - Rango asentamiento: 7.0 - 16.0 cm

📈 DISTRIBUCIÓN DE ASENTAMIENTO:
count    107.000000
mean       9.700935
std        1.802592
min        7.000000
25%        9.000000
50%       10.000000
75%       10.000000
max       16.000000
Name: asentamiento_num, dtype: float64


   - NaN en modulo_finura: 3 → 0
   - Valor promedio usado: 2.09


In [16]:
def entrenar_modelos(X, y):
    """
    Entrena y compara diferentes modelos de ML
    """
    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"   - Entrenamiento: {len(X_train)} muestras")
    print(f"   - Prueba: {len(X_test)} muestras")
    
    # Modelos a probar
    modelos = {
        'Regresión Lineal': LinearRegression(),
        'Árbol de Decisión': DecisionTreeRegressor(max_depth=5, random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
    }
    
    resultados = {}
    
    for nombre, modelo in modelos.items():
        # Entrenar modelo
        modelo.fit(X_train, y_train)
        
        # Predecir
        y_pred = modelo.predict(X_test)
        
        # Evaluar
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        resultados[nombre] = {
            'modelo': modelo,
            'mae': mae,
            'r2': r2,
            'predicciones': y_pred
        }
        
        print(f"   ✅ {nombre}:")
        print(f"      - Error promedio: {mae:.2f} cm")
        print(f"      - R²: {r2:.3f}")
    
    return resultados, X_test, y_test

# Entrenar modelos
resultados, X_test, y_test = entrenar_modelos(X, y)

   - Entrenamiento: 85 muestras
   - Prueba: 22 muestras
   ✅ Regresión Lineal:
      - Error promedio: 0.68 cm
      - R²: -0.394
   ✅ Árbol de Decisión:
      - Error promedio: 0.05 cm
      - R²: 0.932
   ✅ Random Forest:
      - Error promedio: 0.17 cm
      - R²: 0.864


In [17]:

#GRÁFICO 5: Comparación de modelos
modelos_nombres = list(resultados.keys())
mae_scores = [resultados[nombre]['mae'] for nombre in modelos_nombres]
r2_scores = [resultados[nombre]['r2'] for nombre in modelos_nombres]

fig5 = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Error Promedio (MAE)', 'Coeficiente R²'])

fig5.add_trace(go.Bar(x=modelos_nombres, y=mae_scores, 
                     name='MAE', marker_color='#FF6B6B'), 1, 1)
fig5.add_trace(go.Bar(x=modelos_nombres, y=r2_scores, 
                     name='R²', marker_color='#4ECDC4'), 1, 2)

fig5.update_layout(title='Comparación de Modelos de ML', showlegend=False)
fig5.show()


mejor_modelo_nombre = min(resultados.keys(), key=lambda x: resultados[x]['mae'])
mejor_modelo = resultados[mejor_modelo_nombre]



print(f"\n MEJOR MODELO: {mejor_modelo_nombre}")
print(f"   - Error promedio: {mejor_modelo['mae']:.2f} cm")
print(f"   - Precisión (R²): {mejor_modelo['r2']:.3f}")


 MEJOR MODELO: Árbol de Decisión
   - Error promedio: 0.05 cm
   - Precisión (R²): 0.932


In [18]:
def predecir_asentamiento(modelo, caracteristicas):
    """
    Función para predecir asentamiento con nuevos datos
    """
    # Crear DataFrame con las mismas características
    columnas = ['densidad', 'peso_suelto', 'peso_compactado', 'diametro_maximo', 'fck', 'modulo_finura']
    nuevo_dato = pd.DataFrame([caracteristicas], columns=columnas)
    
    # Predecir
    prediccion = modelo.predict(nuevo_dato)[0]
    
    return prediccion

# Ejemplo de predicción
print("\n EJEMPLO DE PREDICCIÓN:")
ejemplo_caracteristicas = [2.6, 1.5, 1.6, 19.0, 250, 2.5]
prediccion_ejemplo = predecir_asentamiento(mejor_modelo['modelo'], ejemplo_caracteristicas)

print(f"   Para una mezcla con:")
print(f"   - Densidad: {ejemplo_caracteristicas[0]}")
print(f"   - Tamaño máximo: {ejemplo_caracteristicas[3]} mm")
print(f"   - fck: {ejemplo_caracteristicas[4]} kg/cm²")
print(f"   - Módulo finura: {ejemplo_caracteristicas[5]}")
print(f"   📏 Asentamiento predicho: {prediccion_ejemplo:.1f} cm")


 EJEMPLO DE PREDICCIÓN:
   Para una mezcla con:
   - Densidad: 2.6
   - Tamaño máximo: 19.0 mm
   - fck: 250 kg/cm²
   - Módulo finura: 2.5
   📏 Asentamiento predicho: 9.0 cm


In [19]:
# Guardar modelo entrenado
import joblib

joblib.dump(mejor_modelo['modelo'], 'modelo_asentamiento.pkl')


# Guardar predicciones
df_predicciones = pd.DataFrame({
    'Real': y_test,
    'Predicho': mejor_modelo['predicciones'],
    'Error': abs(y_test - mejor_modelo['predicciones'])
})
df_predicciones.to_csv('predicciones_asentamiento.csv', index=False)
print("💾 Predicciones guardadas como 'predicciones_asentamiento.csv'")



💾 Predicciones guardadas como 'predicciones_asentamiento.csv'


In [20]:
# GRÁFICO ESPECIAL: Cono de Abrams y Asentamiento

def grafico_cono_abrams(asentamiento_real, asentamiento_predicho=None):
    """
    Crea un gráfico que muestra el cono de Abrams y el asentamiento
    """
    # Dimensiones del cono de Abrams (cm)
    altura_cono = 30
    diametro_superior = 10
    diametro_inferior = 20
    
    fig = go.Figure()
    
    # 1. CONO ORIGINAL (vacío)
    # Puntos para el cono truncado
    x_cono = [0, diametro_inferior/2, diametro_superior/2, 0, -diametro_superior/2, -diametro_inferior/2, 0]
    y_cono = [0, 0, altura_cono, altura_cono, altura_cono, 0, 0]
    z_cono = [0, 0, 0, 0, 0, 0, 0]
    
    fig.add_trace(go.Mesh3d(
        x=x_cono,
        y=y_cono, 
        z=z_cono,
        color='lightblue',
        opacity=0.6,
        name='Cono Vacío'
    ))
    
    # 2. HORMIGÓN DESPUÉS DEL ASENTAMIENTO
    altura_hormigon = altura_cono - asentamiento_real
    
    x_hormigon = [0, diametro_inferior/2, diametro_superior/2, 0, -diametro_superior/2, -diametro_inferior/2, 0]
    y_hormigon = [0, 0, altura_hormigon, altura_hormigon, altura_hormigon, 0, 0]
    z_hormigon = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  # Pequeño desplazamiento en Z
    
    fig.add_trace(go.Mesh3d(
        x=x_hormigon,
        y=y_hormigon,
        z=z_hormigon,
        color='brown',
        opacity=0.8,
        name=f'Hormigón (Asentamiento: {asentamiento_real}cm)'
    ))
    
    # 3. LÍNEA QUE MUESTRA EL ASENTAMIENTO
    fig.add_trace(go.Scatter3d(
        x=[diametro_superior/2 + 2, diametro_superior/2 + 2],
        y=[altura_cono, altura_hormigon],
        z=[0, 0],
        mode='lines+text',
        line=dict(color='red', width=4),
        text=[f'', f'Asentamiento: {asentamiento_real}cm'],
        textposition='middle right',
        name='Medición'
    ))
    
    fig.update_layout(
        title=f'Cono de Abrams - Asentamiento: {asentamiento_real} cm',
        scene=dict(
            xaxis_title='Ancho (cm)',
            yaxis_title='Altura (cm)',
            zaxis_title='',
            camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
        ),
        height=600
    )
    
    return fig

#GRÁFICO 8: Visualización del cono de Abrams
print("\nGrafico de asentamiento - Cono de Abrams")

# Usar un asentamiento de ejemplo o el promedio de los datos
asentamiento_ejemplo = df_ml['asentamiento_num'].mean()

fig8 = grafico_cono_abrams(round(asentamiento_ejemplo, 1))
fig8.show()


Grafico de asentamiento - Cono de Abrams
